# CartoonGAN : Generative Adversarial Networks for Photo Cartoonization
  

# Abstract(논문 개요)
- 사진을 카툰화해주는 생성적 적대적 신경망
- 분야 : computer vision, computer graphics
- 최근들어 이미지를 스타일링하는 것이 유행하고 있다.
- 하지만, 만족스러운 결과를 만들어내지는 못했다.
  - 이유 1. 만화는 높은 수준의 단순화 및 추상화와 함께 독특한 특징들을 가지고 있다.
  - 이유 2. 만화는 엣지가 깔끔하고, 색이 부드럽고, 상대적으로 단순한 텍스쳐를 가지는 경향이 있다. 따라서, 기존 방식인 텍스쳐를 기반으로 한 손실함수로는 상당한 어려움이 존재한다.
- CartoonGAN은 만화 형식으로 스타일링해주는 GAN 모델이다.
- 훈련 데이터 : 짝지어 지지 않은 사진, 만화 이미지
- loss
  1. semantic content loss(의미론적 콘텐츠 손실)
  - 사진과 만화 사이의 상당한 스타일 변화를 다루기 위해 VGG 네트워크의 고수준의 feature map에서 희귀성을 위한 정규화를 만들어낸다.
  2. adversarial loss
  - 깔끔한 엣지를 지키기 위한 엣지를 강조
- 초기화 단계, 타켓 매니폴드(target manifold)를 위한 컨볼루젼 네트워크를 향상시키는 방법 소개


# 1. Introduction
- 만화는 일상 생활에서 널리 활용되고 있다.
- 많은 유명한 만화 이미지는 실제 장면을 기반으로 만들어졌다.
- 실제 장면을 만화 스타일로 수작업하는 것은 매우 힘들고 상당한 예술적 스킬을 포함한다.
- 하지만, 현재 나와있는 소프트웨어나 알고리즘으로는 만족할만한 결과물이 나오지 않는다.
- 따라서 자동으로 실제 장면을 카툰화해주는 것이 생기면 아티스트에게도 도움이 되고, 시간 절약도 되고, 좀 더 창조적인 일에 집중할 수 있게 된다.
- 또한, 인스타그램, 포토샵과 같은 사진 편집 소프트웨어에도 유용한 도움이 된다.
- 예술적 방식으로 이미지를 스타일링 하는 방식은 비사실적 렌더링 분야에서 광범위하게 연구되었다.
- 기존의 방식은 특정 스타일에 대한 전용 알고리즘을 개발한다.
- 하지만, 각각의 아티스트를 모방하는 것은 상당한 노력이 필요하다.
- 최근에는 주어진 이미지를 기반으로 스타일링이 가능한 학습 기반 스타일 변화 방법이 상당한 주목을 받고 있다.
- 특히, 짝지어지지 않은 사진과 스타일된 이미지로부터 훈련된다는 독특한 특징을 가지고 있는 GAN 모델이.
- 학습 기반 스타일 변화 방법은 상당한 성공을 거두었지만 괜찮은 품질의 카툰화된 이미지를 만들어내는 것에는 실패했다.
  - 이유 1. 텍스쳐를 추가하는 것 대신에, 만화는 실제 모습과 비교하여 매우 단순하고 추상적이다.
  - 이유 2. 아티스트들의 다양한 스타일에도 불구하고, 만화는 다른 형태의 예술 작품과는 다르게 눈에띄는 공통적인 특성을 갖는다. - 엣지가 깔끔하고, 부드러운 색상 음영, 상대적으로 단순한 텍스쳐를 갖는다.
- 훈련 데이터 : 사진 데이터셋, 만화 이미지 데이터셋
- 훈련 데이터로부터 고 품질의 결과를 만들어 내기 쉽게 하기 위해서, 사진과 만화 두 데이터셋에서 일치하거나 짝지어지는 데이터를 필요로 하지 않는다.
- computer vision 알고리즘 관점으로부터, 만화 스타일링의 목표는 컨텐츠를 변경하지 않는 동안 여러 개의 사진을 여러 개의 만화로 매핑하는 것이다.
- 이 목표를 달성하기 위해서, 우리는 전용 GAN 기반 architecture를 사용할 것을 제안한다. 간단하면서도 효과적인 손실 함수 2개를 사용하면서.
- 이 논문의 중요한 핵심
  1. 짝지어지지 않은 훈련을 위한 사진, 만화 데이터셋 사용.
  - 최신 기법보다 더 좋은 품질의 카툰화 이미지 출력이 목표.
  - 개개인의 아티스트의 만화를 훈련에 사용하면, 우리의 도구는 그들의 스타일로 재생산(재현)해준다.
  2. GAN기반의 architecture에서 간단하지만 효과적인 2개의 손실 함수를 이용.
  - 생성 신경망에서, 사진과 만화 사이의 상당한 스타일 차이를 다루기 위해서 우리는 의미론적 손실을 VGG 네트워크의 고수준 feature map에서의 $l_{1}$ 희귀성 정규화(sparse reguralization)로 소개하겠다.
  - 판별자 네트워크에서, 우리는 깔끔한 엣지를 위해서, 엣지를 강조하는 적대적 손실을 제안한다.
  3. 현재 존재하는 방법들보다 훨씬 더 효과적인 방법이다.
  - 더 나아가서 타겟 매니폴드로 네트워크가 수렴되는 것을 향상시키기 위한 초기화 단계를 설명하겠다.

> 요점 정리 : 만화는 엣지가 깔끔하고, 부드러운 음영의 특징을 갖는다. CartoonGAN은 짝지어지지 않은 사진, 만화 데이터셋으로 훈련시킨다. 손실함수는 2종류(content loss, adversarial loss). content loss - VGG feature map의 $l_1$ 희소성 정규화. adversarial loss - 엣지 강조. 초기화 단계 중요.

# 2. Related Work(관련 작업)
## 2.1 Non-photorealistic rendering(NPR, 비사실적 렌더링)
- 특정 만화 아티스트들의 스타일을 모방하기 위해서 자동적, 반자동적인 많은 비사실적 렌더링(NPR) 알고리즘들이 개발되었다.
- 일부 작품들은 간단한 음영만으로도 만화효과스러운 3D 모양을 렌더링했다.
- 이러한 "cel shading"(셀 셰이딩) 이라고 불리는 기술들은 아티스트들의 많은 시간을 아껴주고, 만화 비디오와 만화 영화를 만드는데 사용되었다.
- 이미지 필터링, 최적화 방법을 이용한 생성을 이용해서 만화 스타일을 흉내낸 이미지를 만드는 방법들은 많이 나와있다.
- 그러나, 존재하는 사진이나 비디오를 만화처럼 바꾸는 것은 엄청 어려운 일이다.
- 또한, 간단한 수학 공식을 이용해서는 풍부한 예술적 스타일을 포착하기 어렵다.
- 특히, 필터를 추가하거나 최적화 방법으로는 엣지를 깔끔하게 만드는 등과 같이 고차원의 추상적인 아티스트의 행동을 따라하는 건 어렵다.
- 이러한 결과를 개선시키기 위해서, 대체 방법으로 나온 것이 이미지와 비디오를 분할하는 것이다. 하지만, 사용자 상호작용이 필요하다.
- 초상화를 위해 개발된 전용 방법도 있다. 의미론적 분할(semantic segmentation)은 얼굴의 구성요소를 발견하는데서 자동적으로 유래되었다.
- 하지만, 그러한 방법은 일반적인 이미지에 대해서는 잘 대응되지 않는다.

> 요점 정리 : 비사실적 렌더링은 카툰화에 적합한 방법이 아니다.


# 2.2 Stylization with neural networks
- CNN은 많은 computer vision 문제들을 해결하는데 많은 관심을 받았다.
- 각각의 스타일에 상당한 노력이 필요한 비사실적 렌더링(NPR) 알고리즘을 개발하는 것 대신에, style transfer가 활발히 연구되고 있다.
- 짝을 이룬 스타일/비스타일 이미지를 필요로 하는 전통적인 style transfer 방식과는 달리, 최근의 연구에 따르면 객체 인식을 위해 훈련된 VGG 네트워크는 style transfer에서 매우 중요한 객체의 의미론적 특징을 추출하는 능력이 우수하다.
- 결과적으로, 짝을 이룬 이미지가 필요하지 않은 더 강력한 style transfer 방법이 개발되었다는 것이다.
- CNN을 기반으로 한 neural style transfer 방법은 스타일 이미지에서 콘텐츠 이미지로 스타일을 전송시켜준다.
- CNN을 기반으로 한 neural style transfer는 미리 훈련된 VGG 네트워크를 사용한다. 콘텐츠 이미지를 표현하고 결과 이미지를 최적화한다. 스타일 이미지의 텍스쳐 정보를 매칭하면서 콘텐츠 이미지의 콘텐츠를 유지하도록. 여기서 텍스쳐는 global gram matrix를 사용해서 설명된다. 이것은 자동적으로 다양한 예술적인 스타일들로 전송하는 좋은 결과를 만들어 낸다. 하지만, 비슷한 스타일의 content, style image를 필요로 한다. 게다가, 이미지에 여러 객체가 포함되어있으면, 이것은 아마도 의미론적으로 다른 영역으로 스타일을 전송할수도 있다. 만화 스타일 전송의 결과는 엣지를 깔끔하게 만들지 못하고, 부드러운 음영을 재현하지 못하기 때문에 문제가 발생할 수 있다.
- Li와 Wand는 CNN의 feature map의 local matching(지역 정합 방법)에 의해 style transfer를 얻었다. Markov Random Field for fusion(CNNMRF)를 이용해서.
- 그러나, local matching은 의미론적으로 잘못된 출력을 만드는 실수를 할 수 있다.
- Liao 등등은 스타일을 전송하면서 content와 style image 사이에서 의미있는 밀도있는 일치성을 찾을 수 있는 Deep Analogy 방법을 제안했다. 그 방법은 VGG feature space에서 부분 부분을 비교하고 혼합할 수 있다.
- Chen 등등은 만화/비만화 이미지를 분류하기 위해 전용 CNN을 훈련시킴으로써 만화 스타일을 향상시킬 수 있는 방법을 제안했다.
- 이러한 모든 방법들은 style image에서 style, content 를 분리하는 것 어려움 -> 애매모호함
- 그렇기 때문에 결과 이미지는 style image를 뭘 선택하는 지에 따라 크게 달라진다.
- 이와 비교하여 우리의 방법(CartoonGAN)은 실제 사진과 만화 이미지 두 세트를 학습한다.
> 요점 정리 : 다양한 방법들이 있었지만 그 방법들은 전부 style image에서 style과 content를 분리하기에 애매하고 어렵기 때문에 CartoonGAN이 가장 좋은 성능을 낼 수 있다.

# 2.3. Image synthesis with GANs(GAN을 통한 이미지 합성)
- 이미지 합성에 대한 유망한 접근 방식은 GAN(Generative Adversarial Network, 생성적 적대적 신경망)을 사용하는 것이다.
- GAN은 텍스트 -> 이미지로의 번역, 이미지 복원, 이미지 초고해상도 등과 같은 많은 적용분야에 최신 기술을 사용한 결과를 만들어낸다.
- GAN 모델의 핵심 아이디어는 생성자와 판별자라는 2개의 네트워크를 반복하여 훈련시키는 것이다. 판별자에 의한 adversarial loss는 타겟 매니폴드로 생성된 이미지를 밀어낸다.
- 여러 연구에서 pixel-to-pixel 이미지 합성 문제에 대해 GAN 솔루션을 제공했습니다. 하지만, 이러한 방법들은 훈련을 위한 짝지어진 이미지 데이터셋이 필요하다. 그러한 일치하는 이미지 셋을 얻는 것은 어려운 일이기 때문에 만화화를 하기에 쉽지 않은 방법이다.
- 이러한 근본적인 한계를 보내버리기 위해, CycleGAN이라는 방식이 최근에 제안되었다. 짝지어지지 않은 훈련 데이터를 사용하여 이미지 변환을 수행할 수 있게 해주는 프레임워크이다.
- 이 목표를 달성하기 위해서, CycleGAN은 동시에 두 세트의 GAN모델을 학습한다. 그리고 각각 클래스 A에서 클래스 B로, 클래스 B에서 클래스 A로 mapping한다.
- 같은 클래스로 이미지를 매핑하는 결합 매핑에 기반하여 손실 함수값이 구해진다.
- 하지만, 2개의 GAN 모델을 동시에 훈련시키는 것은 종종 느리게 수렴된다. 왜냐하면 훈련 과정에서 시간이 많이 소요되기 때문이다.
- 이 CycleGAN 방법은 또한 카툰 이미지의 특성들(고수준의 추상화와 깔끔한 엣지 등등) 때문에 만화 스타일화하는 것에 안 좋은 결과물들을 만들어낸다.
- 우리의 방법(CartoonGAN)은 짝지어지지 않은 훈련 데이터셋을 이용하여 사진과 만화 매니폴드간의 mapping을 GAN 모델을 활용하여 학습한다.
- 우리의 전용 손실함수 덕분에, 우리의 방법은 고 품질의 카툰 이미지를 합성할 수 있고, 더 효과적으로 훈련가능하다.

> 요점 정리 : 이미지 합성에서 다양한 GAN(pix2pix GAN, CycleGAN)이 활용 -> 만화 이미지의 특성들 잘 표현 X -> CartoonGAN 은 성능 좋음. CartoonGAN 의 훈련 데이터 : 짝지어지지 않은 이미지 데이터셋. 사진, 만화 manifold간의 mapping을 모델이 해줌. 전용 손실함수 덕분에 좋은 성능낼 수 있음.

# 2.4. Network architectures(네트워크 구성)
- 많은 연구들은 이렇게 말한다. 깊은 신경망은 잠재적으로 복잡한 함수를 표현하는 능력을 향상시킬 수 있다. 하지만, 깊은 신경망은 훈련시키기에는 어렵다. 왜냐하면, 악명높은 vanishing gradient problem 때문이다.
- 최근에 도입된 개념인 Residual Block이 훈련 과정을 간단하게 만들기 위해 아주 좋은 선택이 된다.
- Residual Block은 훈련하는 동안 vanishing gradient 문제를 없애주는 "identity shortcut connection"을 설계한다.
- 이러한 Residual Block을 기반으로 한 모델은 생성하는 네트워크에서 엄청난 성능을 보여준다.
- 깊은 CNN을 쉽게 훈련시키기 위해 사용하는 또 다른 방법으로는 배치 정규화가 있다. 배치 정규화는 내부 공변량의 변화에 대응하고, 최소 지점에 접근할 때의 움직임을 줄이기 위해 설계되었다.
- 게다가, Leaky ReLU(LReLU)는 효과적인 gradient propagation(기울기 전파)를 위해 깊은 CNN 신경망의 함수로 널리 사용된다. Leaky ReLU는 유닛이 활성 상태가 아닐 때 0이 아닌 작은 기울기가 들어오게 함으로써 네트워크의 성능을 향상시켰다.
  - 즉, 원래의 ReLU는 유닛으로 들어오는 입력값이 0보다 작을 경우 무시했는 데, Leaky ReLU는 0보다 작은 값도 무시하지 않고 작은 값으로 살려준다.
- 우리는 우리의 카툰화 깊은 구조에 이러한 기술들을 통합시켰다.

> 요점 정리 : CartoonGAN 모델은 vanishing gradient 문제를 해결하기 위해 Residual Block 사용, Batch Normalization 활용, 활성화 함수로 Leaky ReLU를 사용

# 3. CartoonGAN
- GAN 프레임워크는 2개의 CNN으로 구성된다. 
- 하나는 discriminator(판별자)를 속이는 출력을 생성하도록 훈련된 generator(생성자) G이다. 
- 다른 하나는 실제 타겟 매니폴드에서 온 이미지인지 가짜 이미지 인지를 분류해내는 discriminator(판별자) D이다.
- 우리는 generator와 discriminator 네트워크를 만화 이미지의 특성에 적합하도록 설계한다.
- 우리는 실제 사진을 만화 이미지로 변환하는 학습과정을 사진 매니폴드 P를 카툰 매니폴드 C로 매핑하는 매핑 함수로 표현했다.
- 매핑 함수는 훈련 데이터 $S_{data}(p) = \left\{ p_{i} |i = 1 . . . N \right\} ⊂ P, S_{data}(c) = \left\{c_{i} | i = 1,,,M \right\}$ 를 학습한다. 
  - $N$ : 사진 훈련 데이터셋의 갯수, $M$ : 카툰 이미지 훈련 데이터셋의 갯수
- 판별자 함수 D의 목적은 만화 매니폴드를 다른 이미지들과 구별하는 것이다.
- 우리의 목표는 min-max 문제를 해결하는 것이다.
  - $(G^∗ , D^∗ ) = arg min_G max_D \mathcal{L}(G, D)$
- 3.1절 내용 : 네트워크 구조
- 3.2절 내용 : G, D에 대한 2개의 손실 함수
- 3.3절 내용 : 네트워크 수렴을 향상시키기 위해 초기화 단계와 그것을 CartoonGAN에 포함시킨다.

> 요점 정리 : CartoonGAN의 구성요소는 generator(생성자) G와 discriminator(판별자) D이다. 훈련 데이터는 사진과 카툰 이미지이다. 생성자는 실제 사진을 만화 이미지로 변환하는 역할, 판별자는 만화 매니폴드와 다른 이미지를 구별하는 역할을 한다.

# 3.1. CartoonGAN architecture(CartoonGAN 구조)
- CartoonGAN에서 생성자 네트워크 G는 입력 이미지를 만화 매니폴드로 매핑하는데 사용된다. 카툰 스타일화는 모델을 훈련시킬 때 된다.
- 생성자 네트워크 G는 공간을 압축하고 인코딩하기 위해, flat convolution stage에서 시작해서 2개의 down-convolution block까지 이어진다.
- 다운스트림 변환(downstream transformation)을 위한 이 단계에서 유용한 local signal이 추출된다.
- 동일한 레이아웃의 8개 residual block을 content와 매니폴드 feature를 구성하는 데 사용한다.
- 우리는 real-time style transfer 논문에서 제안한 residual block 층을 사용한다.
- <img src="https://aruie.github.io/assets/post/191114-01.png" width=700>

  - CartoonGAN의 생성자, 판별자 네트워크의 구조
  - k : kernel의 크기
  - n : feature map의 수
  - s : convolution layer의 stride
  - norm : normalization layer
  - ES : elementwise sum(요소별 합)
- 카툰 스타일의 이미지는 컨볼루젼 층(stride=1/2)와 마지막 컨볼루젼 층(kernel size=(7, 7))을 포함하는 2개의 up-convolution block에 의해 재구성되어진다.
- 생성자 네트워크를 보완하는 판별자 네트워크 D는 입력 이미지가 실제 만화 이미지 O/X를 판단하는 데 사용된다.
- 모든 이미지 O/X를 판별하는 것보다 카툰 이미지 O/X를 판별하는 것이 덜 힘든 작업이기 때문에, 우리는 판별자 함수 D에서 파라미터 수가 더 적은 조각 수준의 간단한 판별자를 사용할 것이다.
- 객체 분류와는 다르게, 만화 스타일 구별은 이미지의 Local feature에 의존한다.
- 따라서, 판별자 네트워크 D는 얕게 설계된다.
- flat layer가 있는 단계 이후, 네트워크는 해상도를 낮추고 분류를 위한 필수 local feature를 인코딩하기 위해 2개의 strided convolutional block을 사용한다.
- 그 후, feature construction block과 3x3 convolutional layer는 분류 응답을 얻기 위해 사용된다.
- α = 0.2인 Leaky ReLU 함수는 각 normalization layer 이후에 사용된다.
  - α : 0이상의 값.
  - Leaky ReLU 그래프에서 input x가 0보다 작을 때의 직선의 기울기 값의 절댓값.

> 요점 정리 : CartoonGAN 구조 설명

# 3.2. Loss Function(손실 함수)
- 손실 함수 $\mathcal{L}(G, D)$는 2개의 파트로 구성된다.
- 2개의 손실함수를 간단히 더하는 형태로 사용한다.
- <img src="https://ml2blogpost.s3.ap-northeast-2.amazonaws.com/imgs_2019CV/1-%E1%84%89%E1%85%AE%E1%84%89%E1%85%B5%E1%86%A85.png" width=500>
- 1. adversarial loss : $\mathcal{L}_{adv}(G, D)$
  - 원하는 매니폴드 변환을 달성하기 위한 판별자 네트워크를 작동시키는 손실 함수
- 2. content loss : $\mathcal{L}_{con}(G, D)$
  - 만화 스타일화를 하는 동안 이미지 content를 보존하기 위한 손실 함수
- $\mathcal{w}$ : 두 손실 함수의 균형을 맞추기 위한 가중치
- $\mathcal{w}$가 클수록 -> 입력 사진에서 더 많은 content 정보가 유지되므로 더 자세한 질감으로 스타일화된 이미지가 생성된다.
- 논문의 모든 실험 내에서는, $\mathcal{w}$을 10으로 설정하여 style, content 보존의 균형을 잘 맞췄다.

# 3.2.1. Adversarial Loss $\mathcal{L}(G, D)$
- Adversarial Loss는 네트워크 G, D 모두에 적용된다.
- 생성자 네트워크 G의 카툰 변환 과정에 영향을 준다.
- Adversarial Loss값은 생성자 G의 출력 이미지가 만화 이미지처럼 보이는 정도를 나타낸다.
- 이전의 GAN 프레임워크에서의 판별자 D의 역할 : 입력 이미지가 생성자에서 합성되는지/ 실제 타겟 매니폴드에서 합성되는지를 알아내는 것이었다.
- 그러나, 생성된 만화 이미지/ 실제 만화 이미지를 분리하도록 판별자 D를 교육시키는 것만으로는 -> 사진을 만화로 변환하는 데 충분하지 않다. -> 깔끔한 엣지의 표현이 만화 이미지의 중요한 특징이기 때문이기도 하지만, 이러한 엣지의 비율은 일반적으로 전체 이미지 중에서는 매우 작은 비율을 차지하기 때문이다. -> 깔끔한 복제된 엣지가 없지만 정확한 음영을 갖는 출력 이미지가 일반적인 손실함수값(standard loss)에 의해 훈련된 판별자에게 혼동을 줄 수 있다.
- 이 문제를 피하기 위해서, 훈련 만화 이미지로부터 우리는 깔끔한 엣지를 제거한 이미지 데이터셋을 자동적으로 만들어준다.
  - 훈련 만화 이미지 데이터셋 : $S_{data}(c) ⊂ C$
  - 엣지를 제거한 이미지 데이터셋 : $S_{data}(e) = \left\{ e_i |i = 1 . . . M \right\} ⊂ E$
  - $C$ : 깔끔한 엣지가 있는 카툰 매니폴드
  - $E$ : 깔끔한 엣지가 없는 카툰같은 이미지 매니폴드
- 각각의 진짜 이미지 $c_i ∈ S_{data}(c) $에 대한 자세한 단계
  - 1. standard Canny edge detector를 사용하여 엣지 픽셀을 찾는다
  - 2. 엣지 영역을 확장한다.
  - 3. 확장된 엣지 영역에 가우시안 smoothing 방법을 적용한다.
- <img src="https://aruie.github.io/assets/post/191114-02.png" width=500>

  - 만화 이미지 원본, 엣지가 매끄럽게 수정된 버전의 이미지
- 사진 매니폴드 $\mathcal{P}$의 사진 $p_k$ 각각에 대해 생각해보면, 생성자 G는 이미지 $G(p_k)$를 생성한다.
- CartoonGAN에서, 판별자 D를 훈련시키는 것의 목적은 $G(p_k)$에 올바른 label을 할당할 확률을 최대화하는 것이다. 깔끔한 엣지가 없는 만화 이미지(i.e., $e_j ∈ S_{data}(e)$), 실제 만화 이미지(i.e., $c_i ∈ S_{data}(c)$로. 생성자 G가 입력을 정확한 매니폴드로 변환하여 정확하게 안내할 수 있게 해준다.
- 그러므로, adversarial loss를 촉진하는 엣지는 이렇게 정의한다.
- $L_{adv}(G, D) = \mathbb{E}_{c_{i}∼S_{data}(c)} [log D(c_{i})] 
\\ + \mathbb{E}_{e_{j}∼S_{data}(e)} [log(1 − D(e_{j}))] 
\\ + \mathbb{E}_{p_{k}∼S_{data}(p)} [log(1 − D(G(p_{k})))].$

> 요점 정리 : adversarial loss 공식. 엣지 제거를 위해 canny edge detector를 사용하고 gaussian smoothing 방법을 사용한다.

# 3.2.2. Content loss $\mathcal{L}_{con}(G, D)$
- 정확한 매니폴드 사이의 변환 이외에도, 만화 스타일화를 하는 데 한가지 가장 중요한 목표는 입력 사진의 의미론적인 content를 유지하는 만화 이미지를 만드는 것을 확실하게 하는 것이다.
- CartoonGAN에서, 우리는 "O. Russakovsky, J. Deng, H. Su, J. Krause, S. Satheesh, S. Ma, Z. Huang, A. Karpathy, A. Khosla, M. Bernstein 등등 Imagenet large scale visual recognition challenge. International Journal of Computer Vision, 115(3):211–252, 2015." 논문에 의해 사전훈련된 VGG 네트워크에서 고수준의 feature map을 사용한다. 우수한 객체 보존 능력이 입증된.
- 따라서, content loss를 다음과 같이 정의한다.
- $\mathcal{L}_{con}(G, D) = \mathbb{E}_{p_{i}∼S_{data}(p)} [||VGG_{l}(G(p_{i})) − VGG_{l}(p_{i})||_{1}]$
  - $l$ : 특정 VGG layer의 feature map
  - $|| \quad ||_1$ : l1 규제
- 다른 이미지 생성 방법과는 달리, 우리는 의미론적인 content loss를 정의했다. 입력된 사진과 생성된 만화 이미지 사이의 VGG feature map의 $l_1$ 규제를 사용하여. $l_1$ 정규화 방법은 standard $l_2$ norm 방법보다 더 변화에 잘 대응한다.
- 나중에 보여줄 예정이지만, 이러한 변화에 잘 대처하는 것은 만화 스타일을 재생산(재현)하는데 중요하다.
- 우리는 의미론적 content loss을 계산하기 위해 'conv4_4' 층의 feature map을 사용한다.

> 요점 정리 : content loss 공식, 사전훈련된 VGG 네트워크의 feature map 특히 conv4 _4를 사용. l2 norm보단 l1 규제가 더 변화에 잘 대응하기 때문에 l1 규제를 사용한다.


# 3.3. Initialization phase(초기화 단계)
- GAN 모델은 랜덤 초기화를 할 때 매우 비선형이기 때문에, 최적화(optimization)가 쉽게 최선이 아닌 극솟값으로 빠질 수 있다.
- 이러한 수렴을 향상시키는 데 도움을 주기 위해서, 우리는 새로운 초기화 단계를 제안했다.
- 생성자 네트워크 G의 목표는 의미론적인 content를 유지하면서 입력 사진을 만화 스타일로 재구성하는 것이다.
- 입력 이미지의 content만 재구성하는 생성자로 적대적 학습 프레임 워크를 시작한다.
- 초기화 단계에서 의미론적인 content loss $\mathcal{L}_{con}(G, D)$만을 사용하여 생성자 네트워크 G를 사전 훈련시킨다.
- <img src="https://aruie.github.io/assets/post/191114-03.png" width=500>

  - (a) 원래의 사진
  - (b) 초기화 단계를 거친 결과
  - 이 사진은 적절한 재구성을 생성한 초기화 훈련 단계의 10 epoch 이후 재생성된 이미지이다.
- 이 실험 결과는 간단한 초기화 단계가 CartoonGAN 이 성급한 수렴 없이 좋은 배치로 수렴하는 데 도움이 된다는 것을 보여준다.
- style transfer 품질을 향상시키기 위해 결과 이미지를 초기화하는 content 이미지를 사용하는 논문 " L. Gatys, A. Ecker, and M. Bethge. Image style transfer using convolutional neural networks. In IEEE Conference on Computer Vision and Pattern Recognition (CVPR), pages 2414–2423, 2016"에서도 비슷한 결과를 관측할 수 있다.

> 요점 정리 : 랜덤 초기화 단계를 거치지 않으면 최선이 아닌 다른 엉뚱한 극솟값으로 빠질 수 있다. 그러므로 생성자(generator G)는 content loss만을 이용하여 10 epoch만큼 훈련하는 "초기화 단계"를 꼭 거쳐야만 한다.



# 4. Experiments
- 논문에서 구현한 언어 : Torch, Lua
- 논문에서 미리 훈련시켜 놓은 모델을 사용하여 향후 방법을 쉽게 평가할 수 있다.
- 실험 환경 : NVIDIA Titan Xp GPU
- <img src="https://aruie.github.io/assets/post/191114-04.png" width=400>
- CartoonGAN은 각각의 아티스트의 데이터를 훈련에 사용하여 고품질의 카툰 타일을 만들 수 있다. CartoonGAN은 짝지어진 이미지가 필요하지 않기 때문에 만화 비디오에서 쉽게 얻을 수 있다.
- 다른 아티스트들은 그들만의 독특한 만화 스타일을 가지고 있다. CartoonGAN에 이해 효과적으로 학습이 가능한.
- CartoonGAN을 최신 기술과 비교하기 위해 4.1절에 제시된대로 훈련 데이터와 테스트 데이터를 수집했다.
- 4.2절에서는, 제안된 방법과 대표적인 스타일링 방법을 비교하겠다.
- 4.3절에서는, CartoonGAN 모델의 각 구성요소의 효과를 분석하기 위해서 추가 제거 실험(ablation experiment)을 제시하겠다.

# 4.1. Data
- 훈련 데이터 : 실제 사진, 만화 이미지
- 테스트 데이터 : 실제 사진
- 모든 훈련 이미는 resize하고 256x256으로 자른다.
- 사진
  - Flickr 홈페이지에서 6153장 다운로드
  - 5402장의 사진 : 훈련용
  - 751장의 사진 : 테스트용
- 만화 이미지
  - 실제 장면의 만화 이미지를 만들 때마다 아티스트마다 스타일이 다르다
  - 동일한 스타일의 만화 이미지 데이터셋을 얻기 위해서, 훈련 데이터를 동일한 예술가에 의해 만들어지고 그려진 만화 영화의 key frame으로 사용한다.


 논문 속 모델 스타일 | 훈련용 만화 이미지 갯수
 ----- | ----- 
 Makoto Shinkai | 4573개
 Mamoru Hosoda | 4212개
 Spirited Away | 3617개
 Paprika | 2302개

 > 요점 정리 : 훈련 데이터, 테스트 데이터 설명

# 4.2. Comparison with state of the art(최신 기술과 비교)
- 먼저 최근에 제안된 CNN을 기반으로한 스타일화 방법 NST(Neural Style Transfer)와 CycleGAN 2개와 CartoonGAN을 비교해보자.
- original NST 방법
  - 입력값 : 스타일 이미지 1개 ($I_s$), content 이미지 1개 ($I_c$)
  - 스타일 : $I_s$ -> $I_c$로 전송.
  - 공정한 비교를 NST 방법에 2가지 조정을 해줄 수 있다.
    - 1번째 조정 : 입력 사진에 가까운 content가 있는 스타일 이미지를 수동으로 골라준다.
    - 2번째 조정 : NST 방법을 확장하여 훈련을 위해 모든 만화 이미지를 가져온다. (논문 "J.-Y. Zhu, T. Park, P. Isola, and A. A. Efros. Unpaired imageto-image translation using cycle-consistent adversarial networks. In International Conference on Computer Vision, 2017."의 비교 실험과 유사하게.)
- 또한 CycleGAN의 2가지 버전( identity loss ($L_{identity}$)가 있는 버전/ 없는 버전 )을 비교한다.

  > 즉, CartoonGAN, NST + 조정 1번째 모델, NST + 조정 2번째 모델, CycleGAN, CycleGAN + identity loss -> 총 5개의 모델 비교

- 이러한 손실의 혼합은 더 나은 content 보존과 함께 스타일링된 이미지를 생성하는 경향이 있다.
- CycleGAN과 CartoonGAN 두 모델에 대해 200 epoch 훈련.
- <img src="https://aruie.github.io/assets/post/191114-05.png" width=800>
  
  - Gatys라고 적혀있는 것이 NST(Neural Style Transfer) 방법이다. Gatys가 NST 논문의 저자이다.
  - NST 방법과 CycleGAN 방법, CartoonGAN 방법을 비교하는 것이다.
  - 순서대로 원래 사진, 1번째 조정을 해준 NST, 2번째 조정을 해준 NST, CycleGAN, $L_{identity}$가 있는 CycleGAN, CartoonGAN 결과이다.
- NST 방법, CycleGAN 방법은 만화 스타일을 잘 다룰 수 없다는 결과가 사진에 잘 보인다.(Figure 6(b) ~ (e))
- 하지만, 입력된 사진의 content를 보존하면서 필수적인 깔끔한 경계선과 부드러운 음영을 재생산(재현)함으로써, 우리의 CartoonGAN 모델은 고품질의 결과를 만든다.
- style 이미지만을 사용하는 NST 방법은 아마도 style에 대해 완전히 학습하지는 못할 수도 있다. 특히 style 이미지와 content가 다른 영역일 경우.
- NST가 더 많은 훈련 데이터를 얻기 위해 확장된다면, 더 많은 style을 더 잘 배울 수 있다.
- 그러나, 스타일링된 이미지는 불일치하는 인공적인 결과를 만들어냄으로써 local 영역은 다르게 스타일링되는 경향이 있다.
- CycleGAN을 스타일링한 결과는 만화 스타일을 잘 포착해내지 못한다.
- identity loss가 없으면, 출력 결과 이미지는 입력 사진의 content를 잘 보존시켜주지 못한다. (Figure 6(d))
- identity loss는 이러한 문제를 피하기에 유용하다. 하지만, 스타일링된 결과는 여전히 만족스럽진 못하다. (Figure 6(e))
- 하지만, CartoonGAN은 각각의 예술가들의 스타일들을 잘 따라하는 고품질의 카툰화를 만들어낸다.
- <img src="https://aruie.github.io/assets/post/191114-06.png" width=400>
  
  - Figure 7 : Figure 6의 확대본
  - CartoonGAN이 만화 스타일에 매우 중요한 필수적인 모서리를 생성한다는 것을 증명한다.
- CartoonGAN, CycleGAN 둘다 훈련할 때 짝지어지지 않은 데이터가 필요하다.
- 하지만, CartoonGAN 모델이 훈련 시간이 더 적게 소요된다.

모델 | 각 epoch에서 걸리는 시간
----- | -----
identity loss 없는 CycleGAN | 2291.77s
identity loss가 있는 CycleGAN | 3020.31s
CartoonGAN | 1517.69s

- CycleGAN은 양방향 mapping을 위한 2개의 GAN 모델이 필요하기 때문에, 훈련 과정이 엄청 느리다.
- 이미지 카툰화의 경우, 만화에서 사진으로 다시 mapping할 필요가 없다.
- content를 제한하기 위한 cycle architecture 대신에 VGG feature map을 사용함으로써, CartoonGAN은 카툰 스타일을 더 효과적으로 학습할 수 있다.
- 또한, Paprika 스타일과 Mamoru Hosoda 스타일을 CNNMRF 방법, Deep Analogy 방법과 CartoonGAN 방법을 사용하여 비교해보겠다. (Figure 8)
- <img src="https://aruie.github.io/assets/post/191114-07.png" width=800>
  
  - 2가지 방법(CNNMRF, Deep Analogy 방법)은 모두 1개의 스타일 이미지를 기대하기 때문에, 훈련 데이터셋에서 style 이미지를 선택하기 위해 2가지 전략을 사용한다.
  - image 1 : 입력된 사진에 있는 content와 가장 비슷한 만화 이미지를 수동적으로 선택한다.
  - image 2 : 랜덤으로 만화 이미지를 선택한다.
  - 이러한 방법들은 만화 스타일의 특성을 재현하고 명확한 인공적인 결과(artifact)를 만들어내는 데 실패한다. 하지만, CartoonGAN 은 고품질의 스타일화를 만들어낸다.

> 요점 정리 : 최신 기술들(NST, CycleGAN, CNNMRF, Deep Analogy)보다 CartoonGAN의 결과가 더 좋다.

# 4.3. Roles of components in loss function(손실 함수의 구성요소들의 역할)
- CartoonGAN의 각 부분들의 역할을 연구하기 위해서 제거 실험(ablation experiment)을 했다.
- <img src="https://aruie.github.io/assets/post/191114-08.png" width=700>

  - CartoonGAN의 손실함수의 구성요소를 제거하거나/바꾼 결과
  - (a) 입력 사진
  - (b) 초기화 단계 제거했을 때
  - (c) content loss에 $l_2$ 규제를 했을 때
  - (d) adversarial loss에서 edge loss를 제거했을 때
  - (e) CartoonGAN 을 사용했을 때

  - Makoto Shinkai 스타일의 데이터로 훈련된, 손실 함수를 제거한 결과물들.
  - 결과물을 보면 CartoonGAN에서 각각의 구성요소들이 중요한 역할을 하는 것을 보여준다.
  1. 초기화 단계 : 발생자 G가 합리적인 매니폴드로 빠르게 수렴하도록 도와준다.
  - (b) : 초기화 단계가 없을 때, 몇몇의 중요한 feature들은 보이지만, 스타일이 기대와는 거리가 멀다.
  2. 적절한 VGG layer를 사용하더라도, 입력 사진과 style 이미지의 feature map에서의 크고 지역화된 차이가 -> 거대한 style의 차이를 만들기 때문에 여전히 필요하다.(중요하다)
  - 고수준의 VGG feature map의 $l_2$를 대신해서 $l_1$ 희소 정규화를 사용함으로써 사진과 만화 이미지의 상당한 스타일 차이를 대응하는 데 도움을 준다. 
  - 그래서 $l_2$ 규제를 사용하니 원하는 결과가 나오지 않은 것이다.
  3. 정교하게 설계된 edge loss는 생성자 G가 더 나은 만화 스타일의 이미지를 만들수 있게 명확한 엣지를 생성하도록 안내해준다.

# 5. Conclusion and Future Work
- 이 논문에서 우리는 실제 사진을 고품질의 만화 스타일 이미지로 변화하는 생성적 적대 신경망(GAN)인 CartoonGAN을 제안했다.
- 만화 이미지의 특징을 재현하는 것에 초점을 맞춰서, 우리는 2가지를 제안했다.
  1. 명확한 엣지를 위한 새로운 edge에 초점을 맞춘 adversarial loss
  2. content loss를 위한 VGG 신경망의 고수준 feature map의 $l_1$ 희소 정규화. 이것은 부드러운 음영을 재현하기 위한 충분한 유연성을 제공한다.
- 우리는 수렴을 개선하는 데 도움이 되는 간단하면서도 효율적인 초기화 단계를 제안한다.
- 실험은 CartoonGAN 이 실제 사진을 고품질 및 고효율의 만화 스타일 이미지로 변환하는 모델을 학습할 수 있음을 보여주고, 최신의 스타일링 방법들보다 훨씬 더 좋은 결과를 만들어낸다.
- 향후 작업에서는, 초상화의 중요성으로 인해, 우리는 사람의 얼굴을 카툰 스타일화하는 것을 향상시키기 위한 지역적인 얼굴의 특징을 활용하는 방법을 조사하고 싶다.
- 만화 스타일의 특정한 특징들을 다루기 위해서 손실 함수를 설계했다. 하지만, 유사한 아이디어로 다른 이미지 합성 작업에 유용하므로 자세히 더 조사할 것이다.
- 또한, 우리는 영상을 우리의 방법으로 다루기 위해서 훈련 과정에 순차적인 제약을 추가할 예정이다.

---
---
[CartoonGAN 논문 추가 개념 정리](https://github.com/yoonchaiyoung/ML-DL_Study/blob/master/%EB%AA%A8%EB%A5%B4%EB%8A%94%20%EA%B0%9C%EB%85%90%20%EA%B2%80%EC%83%89%ED%95%98%EC%97%AC%20%EC%A0%95%EB%A6%AC.ipynb)

[사진 참조](https://aruie.github.io/2019/11/14/CartoonGAN.html)